In [1]:
from utils import *

In [2]:
rnn_size = 1

data_size = 10
num_data = 10

x_data = []
y_data = []

normalizer = 1

for i in range(num_data):
    input_temp = []
    for j in range(i,i+data_size):
        input_temp.append(j*normalizer)
    x_data.append(input_temp)
    output_temp = normalizer*(i+data_size)
    y_data.append(output_temp) 

x_data = np.array(x_data, dtype = np.float32)
x_data = np.reshape(x_data, [num_data,data_size,1])
y_data = np.array(y_data, dtype = np.float32)
y_data = np.reshape(y_data,[-1,1])

print(x_data.shape)
print(y_data.shape)

(10, 10, 1)
(10, 1)


In [3]:
train_x = tf.placeholder('float', [None, data_size,1 ])
train_y = tf.placeholder('float', [None,1])

train_x_temp = tf.transpose(train_x, [1,0,2])
train_x_temp = tf.reshape(train_x_temp, [-1,1])
train_x_temp =tf.split(train_x_temp, num_or_size_splits=int(data_size), axis=0)

In [4]:
sess = tf.Session()
saver = tf.train.import_meta_graph('./save/rnn.meta')
saver.restore(sess, './save/rnn')
weight = get_tensor_by_name("Variable:0")
bias = get_tensor_by_name("Variable_1:0")
print_keys("trainable_variables")

Collection name : trainable_variables
Tensor("Variable/read:0", shape=(1, 1), dtype=float32)
Tensor("Variable_1/read:0", shape=(1,), dtype=float32)
Tensor("rnn/rnn/lstm_cell/weights/read:0", shape=(2, 4), dtype=float32)
Tensor("rnn/rnn/lstm_cell/biases/read:0", shape=(4,), dtype=float32)


In [5]:
lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size) 
print_keys("trainable_variables")


Collection name : trainable_variables
Tensor("Variable/read:0", shape=(1, 1), dtype=float32)
Tensor("Variable_1/read:0", shape=(1,), dtype=float32)
Tensor("rnn/rnn/lstm_cell/weights/read:0", shape=(2, 4), dtype=float32)
Tensor("rnn/rnn/lstm_cell/biases/read:0", shape=(4,), dtype=float32)


In [6]:
with tf.variable_scope("restore") as scope:
    lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size)
    lstm_weight = tf.get_collection("trainable_variables")[2]
    lstm_bias = tf.get_collection("trainable_variables")[3]
    tf.get_variable(name = 'rnn/lstm_cell/weights'
                    , shape = lstm_weight.get_shape()
                    , initializer = tf.constant_initializer(sess.run(lstm_weight)))
    tf.get_variable(name = 'rnn/lstm_cell/biases'
                    , shape = lstm_bias.get_shape()
                    , initializer = tf.constant_initializer(sess.run(lstm_bias)))
    scope.reuse_variables()
    train_outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, train_x_temp, dtype = tf.float32)
    train_output = tf.matmul(train_outputs[-1], weight) + bias

In [8]:
sess.run(master_initializer(sess))

In [9]:
print(sess.run(train_output, feed_dict = {train_x : x_data}))
print(y_data)

[[ 10.28840828]
 [ 10.8922596 ]
 [ 11.73054886]
 [ 12.85735607]
 [ 14.18762493]
 [ 15.3575592 ]
 [ 16.03669167]
 [ 16.31197357]
 [ 16.40358734]
 [ 16.4320507 ]]
[[ 10.]
 [ 11.]
 [ 12.]
 [ 13.]
 [ 14.]
 [ 15.]
 [ 16.]
 [ 17.]
 [ 18.]
 [ 19.]]


In [10]:
print_keys("trainable_variables")

Collection name : trainable_variables
Tensor("Variable/read:0", shape=(1, 1), dtype=float32)
Tensor("Variable_1/read:0", shape=(1,), dtype=float32)
Tensor("rnn/rnn/lstm_cell/weights/read:0", shape=(2, 4), dtype=float32)
Tensor("rnn/rnn/lstm_cell/biases/read:0", shape=(4,), dtype=float32)
Tensor("restore/rnn/lstm_cell/weights/read:0", shape=(2, 4), dtype=float32)
Tensor("restore/rnn/lstm_cell/biases/read:0", shape=(4,), dtype=float32)
